## Reglering av inomhustemperatur

Vi skall studera ett system för reglering av inomhustemperatur (exemplet är hämtat ur Glad, Ljung: Reglerteknik). Exemplet illustrerar hur man kan bygga ett reglersystem genom att kombinera flera regulatorer i en **kaskadreglering** eller kombinera återkoppling med **framkopppling** från mätbara störningar.

Uppvärmningen av huset antas ske från ett fjärrvärmenät, som via en värmeväxlare levererar varmt vatten till huset. I värmeväxlaren styrs *tilloppstemperaturen* till huset via en blandningsventil, som blandar uppvärmt vatten med kallare returvatten från huset. Den enkla modellen ger tilloppstemperaturen $T_t$ från differentialekvationen 
$$
    \tau T_t(t) = -T_t(t) + T_r + (T_h-T_r)u(t),
$$
där $u$ är styrsignalen till ventilen och $\tau$ är en snabb tidskonstant för blandningsförloppet. Temperaturerna för uppvärmt vatten ($T_h$) och returvatten ($T_r$) approximeras som konstanta. Sambandet ovan beskrivs av överföringsfunktionen $G_1(s)$.

Uppvärmningen av huset beskrivs av modellen
$$
    \dot{T}(t) = \alpha_1 (T_t(t)-T(t)) - \alpha_2 (T(t)-T_{ute}(t)),
$$
där den första termen beskriver uppvärmningen från husets radiatorer, och den andra representerar värmeförluster till omgivningen. Med överföringsfunktioner blir detta
$$
    T(s) = G_2(s) T_t(s) + G_3(s)T_{ute}(s)
$$
där 
$$
    G_2(s) = \frac{\alpha_1}{s+\alpha_1+\alpha_2}, \quad G_3(s)=\frac{\alpha_2}{s+\alpha_1+\alpha_2}.
$$

### Fall 1: Enkel temperaturreglering 
Med en mätning av inomhustemperaturen används först en PI-regulator för att styra blandningsventilen:
$$
    U(s) = \big(0.1+\frac{0.1}{s}\big)\cdot\big(T^{ref}(s)-T(s)\big)
$$
För att testa hur denna reglering fungerar, så antas temperaturen hos det uppvärmda vattnet ($T_h$) sjunka från $70^\circ$ till $30^\circ$ under 2 timmar. Dessutom faller utetemperaturen från $10^\circ$ till $0^\circ$ efter 7 timmar (de något drastiska störningarna har syftet att illustrera effekterna tydligare). 

Koden nedan utgår från en tillståndsmodell för det slutna systemet, där tillståndsvariablerna är
$$
    x_1(t) = T_t(t), \quad x_2(t) = T(t), \quad x_3(t) = \int_0^t (T^{ref}(\tau)-T(\tau))d\tau.
$$

In [ ]:
import numpy as np
import control as ct
import matplotlib as mpl
import matplotlib.pyplot as plt

alpha1 = 0.25
alpha2 = 0.25
tau = 0.01
T_ref = 20.
T_r = 20.
kp = 0.1
ki = 0.1

def hustemp1_rhs(t, x, u, params):
    T_t = x[0]
    T = x[1]
    ui = x[2]   # regulatorns integratortillstånd

    if t<1 or t>3: 
        T_h = 70.
    else:
        T_h = 30

    T_ute = 10.
    if t>7.0: T_ute = 0

    e = T_ref - T
    u1 = kp*e + ki*ui
  
    dT_t = (T_r + (T_h-T_r)*u1 - T_t)/tau
    dT = -(alpha1 + alpha2)*T + alpha1*T_t + alpha2*T_ute
    dui = e
    return [dT_t, dT, dui]

def hustemp1_out(t, x, u, params):
    T_t = x[0]
    T = x[1]
    ui = x[2]

    e = T_ref - T
    u1 = kp*e + ki*ui
    return [T_t, T, u1]

hustemp1 = ct.NonlinearIOSystem(hustemp1_rhs, outfcn=hustemp1_out, inputs=('u'), states=('T_t', 'T', 'ui'), outputs=('T_t', 'T', 'u1'), name='hustemp1')

time = np.linspace(0, 13, 200)
x0 = [30, 20, 2]
t, y = ct.input_output_response(hustemp1, time, 0, x0)

fig, ax = plt.subplots(2, 1)
plt.suptitle('Enkel reglering av inomhustemperatur')
fig.supxlabel('t')
plt.subplot(2, 1, 1)
plt.plot(t, y[1])
plt.ylabel('Inomhustemp $T$ [C]')
plt.subplot(2, 1, 2)
plt.plot(t, y[2])
plt.ylabel('Styrsignal $u$')
plt.show()

### Fall 2: Temperaturreglering i kaskad 
En nackdel med reglersystemet ovan är att det inte kompenserar för den sänkta temperaturen från värmeväxlaren förrän inomhustemperaturen börjat falla. Genom att utnyttja ytterligare en temperaturgivare, som mäter tilloppstemperaturen $T_t$, kan en regulator i en *inre loop* snabbare kompensera för förändringar hos temperaturen $T_h$ för det uppvärmda vattnet. 

Med denna *kaskadreglering* är nu styrsignalen för PI-regulatorn i den *yttre loopen* börvärdet $T_t^{ref}$ för den inre loopen (istället för den tidigare styrsignalen $u$):
$$
    U_2(s) = T_t^{ref}(s) = \big(5+\frac{5}{s}\big)\cdot\big(T^{ref}(s)-T(s)\big)
$$
I den inre kretsen används en annan PI-regulator för att få den önskade tilloppstemperaturen:
$$
    U_1(s) = U(s) = \big(10+\frac{1}{s}\big)\cdot\big(T_t^{ref}(s)-T_t(s)\big)
$$

Tillståndsmodellen för det slutna systemet måste nu utökas, och tillståndsvariablerna är
$$
    x_1(t) = T_t(t), \quad x_2(t) = T(t), \quad x_3(t) = \int_0^t (T^{ref}(\tau)-T(\tau))d\tau, \quad x_4(t) = \int_0^t (T_t^{ref}(\tau)-T_t(\tau))d\tau.
$$

In [ ]:
kp1 = 0.1
ki1 = 10
kp2 = 5
ki2 = 5
tau = 0.01

def hustemp2_rhs(t, x, u, params):
    T_t = x[0]
    T = x[1]
    ui2 = x[2]  # integratortillstånd för regulator #2 (yttre)
    ui1 = x[3]  # integratortillstånd för regulator #1 (inre)

    if t<1 or t>3: 
        T_h = 70.
    else:
        T_h = 30

    T_ute = 10.
    if t>7.0: T_ute = 0

    e2 = T_ref - T
    u2 = kp2*e2 + ki2*ui2
    e1 = u2 - T_t
    u1 = kp1*e1 + ki1*ui1
    if u1>1:    # begränsa styrsignalen till ventilen 
        u1 = 1
    #u1 = (u2 - T_r)/(T_h-T_r)   # korrekt styrsignal kan beräknas om T_h och T_r mäts

    dT_t = (T_r + (T_h-T_r)*u1 - T_t)/tau
    dT = -(alpha1 + alpha2)*T + alpha1*T_t + alpha2*T_ute
    dui2 = e2
    dui1 = e1
    
    return [dT_t, dT, dui2, dui1]

def hustemp2_out(t, x, u, params):
    T_t = x[0]
    T = x[1]
    ui2 = x[2]  # integratortillstånd för regulator #2 (yttre)
    ui1 = x[3] 

    if t<1 or t>3: 
        T_h = 70.
    else:
        T_h = 30
        
    e2 = T_ref - T
    u2 = kp2*e2 + ki2*ui2
    e1 = u2 - T_t
    u1 = kp1*e1 + ki1*ui1
    if u1>1:    # begränsa styrsignalen till ventilen
        u1 = 1
    #u1 = (u2 - T_r)/(T_h-T_r)  # korrekt styrsignal kan beräknas om T_h och T_r mäts
    return [T_t, T, u1]

hustemp2 = ct.NonlinearIOSystem(hustemp2_rhs, outfcn=hustemp2_out, inputs=('u'), states=('T_t', 'T', 'ui2', 'ui1'), outputs=('T_t', 'T', 'u1'), name='hustemp2')

time = np.linspace(0, 13, 400)
x0 = [30, 20, 30/ki2, 0.2/ki1]
t, y = ct.input_output_response(hustemp2, time, 0, x0)

fig, ax = plt.subplots(2, 1)
plt.suptitle('Kaskadreglering av inomhustemperatur')
fig.supxlabel('t')
plt.subplot(2, 1, 1)
plt.plot(t, y[1])
plt.ylabel('Inomhustemp $T$ [C]')
plt.subplot(2, 1, 2)
plt.plot(t, y[2])
plt.ylabel('Styrsignal $u$')
plt.show()

### Fall 3: Temperaturreglering i kaskad och framkoppling från utetemperatur
För att ytterligare förbättra reglersystemets prestanda, så kan man utnyttja en framkoppling från uppmätt utetemperatur. Fördelen med detta är att tilloppstemperaturen kan höjas och kompensera för ett temperaturfall utomhus, innan detta har slagit igenom på inomhustemperaturen. 

När framkopplingen skall beräknas, så kan det slutna systemets överföringsfunktion för den *inre* loopen, dvs från $T_t^{ref}$ till $T_t$, approximeras till 1. Som framgår av kompendiet, blir följden av detta att framkopplingen består av en konstant
$$
    F_f(s) = - \frac{\alpha_1}{\alpha_2}
$$ 
I koden nedan är enda förändringen att framkopplingstermen lagts till i beräkningen av $u_2=T_t^{ref}$.

In [ ]:
kp1 = 0.1
ki1 = 10
kp2 = 5
ki2 = 5
tau = 0.01
kff = alpha2/alpha1 # korrekt framkopplingsparameter

def hustemp3_rhs(t, x, u, params):
    T_t = x[0]
    T = x[1]
    ui2 = x[2]  # integratortillstånd för regulator #2 (yttre)
    ui1 = x[3]  # integratortillstånd för regulator #1 (inre)

    if t<1 or t>3: 
        T_h = 70.
    else:
        T_h = 30

    T_ute = 10.
    if t>7.0: T_ute = 0

    e2 = T_ref - T
    u2 = kp2*e2 + ki2*ui2 - kff*T_ute
    e1 = u2 - T_t
    u1 = kp1*e1 + ki1*ui1
    if u1>1:
        u1 = 1
    #u1 = (u2 - T_r)/(T_h-T_r)   # korrekt styrsignal kan beräknas om T_h och T_r mäts

    dT_t = (T_r + (T_h-T_r)*u1 - T_t)/tau
    dT = -(alpha1 + alpha2)*T + alpha1*T_t + alpha2*T_ute
    dui2 = e2
    dui1 = e1
    
    return [dT_t, dT, dui2, dui1]

def hustemp3_out(t, x, u, params):
    T_t = x[0]
    T = x[1]
    ui2 = x[2]  # integratortillstånd för regulator #2 (yttre)
    ui1 = x[3] 

    if t<1 or t>3: 
        T_h = 70.
    else:
        T_h = 30

    T_ute = 10.
    if t>7.0: T_ute = 0
        
    e2 = T_ref - T
    u2 = kp2*e2 + ki2*ui2 - kff*T_ute
    e1 = u2 - T_t
    u1 = kp1*e1 + ki1*ui1
    if u1>1:
        u1 = 1
    #u1 = (u2 - T_r)/(T_h-T_r)  # korrekt styrsignal kan beräknas om T_h och T_r mäts
    return [T_t, T, u1]

hustemp3 = ct.NonlinearIOSystem(hustemp3_rhs, outfcn=hustemp3_out, inputs=('u'), states=('T_t', 'T', 'ui2', 'ui1'), outputs=('T_t', 'T', 'u1'), name='hustemp3')

time = np.linspace(0, 13, 400)
x01 = [30, 20, (30+kff*10)/ki2, 0.2/ki1]
t1, y1 = ct.input_output_response(hustemp3, time, 0, x01)
kff = 1.25*alpha2/alpha1    # vad händer med en felaktig framkoppling (25% fel)?
x02 = [30, 20, (30+kff*10)/ki2, 0.2/ki1]
t2, y2 = ct.input_output_response(hustemp3, time, 0, x02)

fig, ax = plt.subplots(2, 1)
plt.suptitle('Kaskadreglering och framkoppling')
fig.supxlabel('t')
plt.subplot(2, 1, 1)
plt.ylim((19, 21))
plt.plot(t1, y1[1])
plt.plot(t2, y2[1])
plt.ylabel('Inomhustemp $T$ [C]')
plt.legend(['Med perfekt framkoppling', r'Med 25% fel i framkopplingen'])
plt.subplot(2, 1, 2)
plt.plot(t, y[2])
plt.ylabel('Styrsignal $u$')
plt.show()